In [2]:
import pandas as pd
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.llms.base import LLM

In [ ]:
load_dotenv()

In [ ]:
source = pd.read_parquet('../../data/processed/cleanedData.parquet')

In [ ]:
class openRouterLLM(LLM):
    model: str
    key: str

    def _call(self, prompt, stop=None, run_manager=None):
        client = OpenAI(
            base_url="https://openrouter.ai/api/v1",
            api_key=self.key,
        )
        
        completion = client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}]
        )
        return completion.choices[0].message.content
    
    @property
    def _llm_type(self):
        return "openrouter"    